<a href="https://colab.research.google.com/github/MaryamNourii/Knowledge-Tracing_Recommendation-system-/blob/main/RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/AI/Recommendation_system/KT1/KT1

/content/drive/MyDrive/AI/Recommendation_system/KT1/KT1


In [ ]:
path=r'/content/drive/MyDrive/AI/Recommendation_system/KT1/KT1'
d=[]
table_list=[]
s=pd.Series(os.listdir(path))
file_selected=s.sample(5000).to_numpy()
for file_name in file_selected:
    data_raw=pd.read_csv(path+'/'+file_name,encoding = "ISO-8859-15")
    data_raw['user_id']=pd.Series([file_name[:-4]]*len(data_raw))
    d.append([file_name[:-4],len(data_raw)])
    data=pd.DataFrame(data_raw,columns=['user_id']+data_raw.columns.to_list()[:-1])
    table_list.append(data)
df=pd.concat(table_list)
pd.set_option('display.max_rows',10)
df=df.reset_index(drop=True)
df

,user_id,timestamp,solving_id,question_id,user_answer,elapsed_time
0,u563619,1554960995353,1,q6124,d,24000
1,u563619,1554961019266,2,q10028,a,20000
2,u563619,1554961402528,3,q336,c,17000
3,u563619,1554961422119,4,q4730,d,17000
4,u563619,1554961461735,5,q9580,b,36000
...,...,...,...,...,...,...
671810,u25687,1501155801231,207,q3412,c,24333
671811,u25687,1503281479275,208,q7597,c,12250
671812,u25687,1503281479324,208,q7595,c,12250
671813,u25687,1503281479338,208,q7596,a,12250


In [ ]:
unique_users = df['user_id'].unique()
unique_questions = df['question_id'].unique()

In [ ]:
user2idx = {user: idx for idx, user in enumerate(unique_users)}
question2idx = {question: idx for idx, question in enumerate(unique_questions)}

In [ ]:
df['user_id'] = df['user_id'].apply(lambda x: user2idx[x])
df['question_id'] = df['question_id'].apply(lambda x: question2idx[x])

In [ ]:
df

,user_id,timestamp,solving_id,question_id,user_answer,elapsed_time
0,0,1554960995353,1,0,d,24000
1,0,1554961019266,2,1,a,20000
2,0,1554961402528,3,2,c,17000
3,0,1554961422119,4,3,d,17000
4,0,1554961461735,5,4,b,36000
...,...,...,...,...,...,...
671810,4999,1501155801231,207,92,c,24333
671811,4999,1503281479275,208,11280,c,12250
671812,4999,1503281479324,208,11278,c,12250
671813,4999,1503281479338,208,11279,a,12250


In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2)

In [ ]:
model = Sequential()
model.add(Embedding(len(unique_questions), 64, input_length=100))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_sequences = []
train_labels = []
for user, group in train_data.groupby('user_id'):
    questions = group['question_id'].tolist()
    is_correct = group['is_correct'].tolist()
    train_sequences.append(questions)
    train_labels.append(is_correct)

test_sequences = []
test_labels = []
for user, group in test_data.groupby('user_id'):
    questions = group['question_id'].tolist()
    is_correct = group['is_correct'].tolist()
    test_sequences.append(questions)
    test_labels.append(is_correct)

train_sequences = pad_sequences(train_sequences, maxlen=100, padding='post', value=-1)
test_sequences = pad_sequences(test_sequences, maxlen=100, padding='post', value=-1)

In [ ]:
model.fit(train_sequences, np.array(train_labels), epochs=10, batch_size=32, validation_data=(test_sequences, np.array(test_labels)))

In [ ]:
test_loss, test_acc = model.evaluate(test_sequences, np.array(test_labels))
print('Test accuracy:', test_acc)